In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold, cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load all datasets in numpy array format 
def removeTmpFiles(path):
    # automatically remove all .DS_Store files
    if path.split("/")[-1] == '.DS_Store': os.remove(path)
    elif os.path.isdir(path):
        for filename in os.listdir(path): removeTmpFiles(path + "/" + filename)

data_dir = "data/final_output_data/"
data_train, data_val, data_test = [], [], []
labels_train, labels_val, labels_test = [], [], []

data = {'train': data_train,
        'val': data_val,
        'test': data_test}
labels = {'train': labels_train,
          'val': labels_val,
          'test': labels_test}

dataset_types = ['train', 'val', 'test']
for t in dataset_types: removeTmpFiles(data_dir+t) # for macOS
for t in dataset_types:
    curr_data_dir = data_dir+t
    for folder_name in os.listdir(curr_data_dir):
        folder_path_class = os.path.join(curr_data_dir, folder_name)
        label = folder_name
        for image_path in os.listdir(folder_path_class):
            image_path_full = os.path.join(folder_path_class, image_path)
            image = cv2.imread(image_path_full)
            image = cv2.resize(image, (240, 240))
            image = image.flatten()
            data[t].append(image)
            labels[t].append(label)
    data[t] = np.array(data[t], dtype=object)
    labels[t] = np.array(labels[t], dtype=object)

for t in dataset_types: 
    print("dataset type: ", t)
    print(data[t])
    print(data[t].shape)
    print(labels[t])
    print(labels[t].shape)

In [ ]:
# SVM with cross-validation
def SVC_crossval(X_train, y_train, kernels, C_vals):
    kf = KFold(n_splits=3)
    best_acc, best_kernel, best_c_val, mean_acc = -1, -1, -1, -1
    for i in kernels:
        for j in C_vals:
            model = SVC(kernel=i, C=j)
            acc_folds = []
            for (train_idx, val_idx) in kf.split(X=X_train, y=y_train):
                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
                model.fit(X_fold_train, y_fold_train)
                preds = model.predict(X_fold_val)
                acc = sum([preds == y_fold_val])/len(y_fold_val)
                acc_folds.append(acc)
            mean_acc = np.mean(acc_folds)
            if mean_acc > best_acc:
                best_acc = mean_acc
                best_c_val = j
                best_kernel = i
    return best_kernel, best_c_val, mean_acc

In [ ]:
def test_SVC_model(kernel, C_val, X_train, y_train, X_test):
    model = SVC(kernel=kernel,C=C_val)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [ ]:
y_pred_SVM = test_SVC_model('rbf',10,X_train,y_train,X_test)
print("Classfication report for SVM:")
print(classification_report(y_test, y_pred_SVM, target_names=['']]))

In [ ]:
def KNN(X_train,y_train,X_test):
    knn_model = KNeighborsClassifier(n_neighbors=1)
    knn_model.fit(X_train,y_train)
    y_pred = knn_model.predict(X_test)
    return y_pred

In [ ]:
y_preds_KNN = KNN(X_train,y_train,X_test)
acc = accuracy_score(y_test, y_preds_KNN)
print ("Your model's accuracy on the test set is {:.3f}".format(acc))

In [ ]:
def tree_model(X_train,X_test,y_train,y_test):
    print (X_train)
    dtree_model = DecisionTreeClassifier(random_state=0)
    dtree_model.fit(X_train,y_train)
    pred=dtree_model.predict(X_test)
    test_acc=sum(pred == y_test)/len(y_test)
    return(test_acc)

In [ ]:
tree_acc = tree_model(X_train,X_test,y_train,y_test)
print("Your tree's test set accuracy is {:.3f}".format(tree_acc))

In [ ]:
def rf(X_train,X_test,y_train,y_test):    
    rf = RandomForestClassifier(min_samples_leaf=10,max_features=1,random_state=0)
    rf.fit(X_train,y_train)
    pred_rf=rf.predict(X_test)
    test_acc=sum(pred_rf == y_test)/len(y_test)
    return(test_acc)

In [ ]:
rf_acc = rf(X_train,X_test,y_train,y_test)
print("Your tree's test set accuracy is {:.3f}".format(rf_acc))